In [1]:
#include<iostream>
#include<vector>
#include<math.h>
#include<fstream>
using namespace std;

In [2]:
class Point{
    private:
        float x=0.0,y=0.0;
        int colorRGB[3]={0,0,0};
        
    public:
        Point(float x_coord,float y_coord):x(x_coord),y(y_coord){}

        void info(){
            cout<<"POINT("<<this->x<<","<<this->y<<","<<"["<<
                this->colorRGB[0]<<","<<this->colorRGB[1]<<","<<this->colorRGB[2]<<"])"<<endl;
        }

        float get_x(){
            return this->x;
        }
        float get_y(){
            return this->y;
        }
    
        void set_x(float value){
            this->x = value;
        }
        void set_y(float value){
            this->y = value;
        }
    
    
        int* get_color(){
            return this->colorRGB;
        }
    
        bool operator==(const Point &point) const {
            return x == point.x &&
                   y == point.y;
        }
        bool operator!=(const Point &point) const {
            return !(point == *this);
        }
    
        float getDistance(Point &p) {
            return sqrt(pow(p.x - this->x, 2) + pow(p.y - this->y, 2));
        }
        
        friend std::ostream &operator<<(std::ostream &os, const Point &point){
            os << "(" << point.x << "," << point.y << ")";
            return os;
        }
}
       

In [3]:
class Clust{
    public:
        vector<Point> points;
        vector<int> index_points;
        
        enum COLOR{
            ROJO=0,VERDE,AZUL
        };
        COLOR color = COLOR::ROJO;
        Point centroid = Point(0,0);
        float deviation_x=0.0,deviation_y=0.0;
        
        void update_centroid(){
            for(vector<Point>::iterator point = this->points.begin(); point!=this->points.end();point++){
                centroid.set_x(centroid.get_x()+point->get_x());
                centroid.set_y(centroid.get_y()+point->get_y());
            }
            centroid.set_x(centroid.get_x()/(float)this->points.size());
            centroid.set_y(centroid.get_y()/(float)this->points.size());
        }
    
        void update_deviation(){
            float value_x = 0.0, value_y = 0.0;
            for(vector<Point>::iterator it = this->points.begin(); it!=this->points.end();it++){
                value_x += pow(it->get_x()-centroid.get_x(),2);
                value_y += pow(it->get_y()-centroid.get_y(),2);
            }
            this->deviation_x = pow(value_x/points.size(),0.5);
            this->deviation_y = pow(value_y/points.size(),0.5);
        } 
    
    
        Clust(Point &&centroid, COLOR color){
            this->color = color;
            this->centroid = centroid; 
        }
            
        void add_point(Point &&point){
            this->points.push_back(point);
            this->update_centroid();
            this->update_deviation();
        }
        void add_point(Point &point){
            this->points.push_back(point);
            this->update_centroid();
            this->update_deviation();
        }
        
        void show_points(){
            for(vector<Point>::iterator it = this->points.begin(); it!=this->points.end();it++){
                it->info();
            }    
        }
        void show_stats(){
            cout<<"centroid_Point("<<this->centroid.get_x()<<","<<this->centroid.get_y()<<")"<<endl;
            cout<<"Deviation_Values("<<this->deviation_x<<","<<this->deviation_y<<")"<<endl;
        }
        void delete_points(){
            this->index_points.clear();
            this->points.clear();
        }
        string getPoints() {
            string s = "";
            for (int p: index_points) {
                s += to_string(p) + " ";
            }
            return s;
        }
     
        
}      

In [4]:
// PREPARANDO NUESTROS DATOS

vector<Clust> clusters{{Point(5,19),Clust::COLOR::ROJO},
                       {Point(10,9),Clust::COLOR::VERDE},
                       {Point(12,4),Clust::COLOR::AZUL}};




In [5]:
vector<Point> points{};


In [6]:

void carga(int &&x_max, int &&y_max ,vector<Point> &points){
    int x = 0, y = 0;
    ofstream puntosTXT;
    puntosTXT.open("puntos-500.txt");
    for(int instancia_n = 500 ; instancia_n > 0; instancia_n--){
        x = rand()%x_max;
        y = rand()%y_max;
        puntosTXT<<x<<","<<y<<endl;
        points.push_back(Point(x,y));
    }
    puntosTXT.close();
    cout<<"DONE"<<endl;   
}


In [7]:
carga(500,500,points);


DONE


In [8]:
// MAIN

void populateClusters() {
    for (int p = 0; p < points.size(); ++p) {
        float smallestDistance = clusters[0].centroid.getDistance(points[p]);
        int clusterToJoin = 0;
        for (int i = 1; i < clusters.size(); i++) {
            float distance = clusters[i].centroid.getDistance(points[p]);
            if (distance < smallestDistance) {
                smallestDistance = distance;
                clusterToJoin = i;
            }
        }
        clusters[clusterToJoin].index_points.push_back(p);
    }
}



In [9]:
void MAIN(){
    int iteracion = 1;
    ofstream centroidsFile;
    centroidsFile.open("centroides.txt");
    while(iteracion <= 10){
        populateClusters(); 
        for(int cluster_index = 0; cluster_index < clusters.size(); cluster_index++){
            for(int index : clusters[cluster_index].index_points){
                clusters[cluster_index].add_point(points[index]);
            }
            centroidsFile <<  clusters[cluster_index].centroid << "|-|"<< clusters[cluster_index].getPoints() << endl;
            clusters[cluster_index].delete_points();
        } 
        iteracion++;
    }
    centroidsFile.close();

}

In [10]:
MAIN();